In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/paralel-translation-corpus-in-22-languages/EN-PL/EN-PL.txt
/kaggle/input/paralel-translation-corpus-in-22-languages/EN-SL/EN-SL.txt
/kaggle/input/paralel-translation-corpus-in-22-languages/Sample TMX file (EN-GA)/EN-GA.tmx
/kaggle/input/paralel-translation-corpus-in-22-languages/EN-SV/EN-SV.txt
/kaggle/input/paralel-translation-corpus-in-22-languages/EN-BG/EN-BG.txt
/kaggle/input/paralel-translation-corpus-in-22-languages/EN-FR/EN-FR.txt
/kaggle/input/paralel-translation-corpus-in-22-languages/EN-RO/EN-RO.txt
/kaggle/input/paralel-translation-corpus-in-22-languages/EN-ET/EN-ET.txt
/kaggle/input/paralel-translation-corpus-in-22-languages/EN-GA/EN-GA.txt
/kaggle/input/paralel-translation-corpus-in-22-languages/EN-PT/EN-PT.txt
/kaggle/input/paralel-translation-corpus-in-22-languages/EN-DA/EN-DA.txt
/kaggle/input/paralel-translation-corpus-in-22-languages/EN-MT/EN-MT.txt
/kaggle/input/paralel-translation-corpus-in-22-languages/EN-IT/EN-IT.txt
/kaggle/input/paralel-translation

In [3]:
import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd

In [4]:
CD = '../input/paralel-translation-corpus-in-22-languages/' #initially there were 22 languages, two more added afterwards :)
SL = 'EN' #this is a constant and should not be changed, i.e. Source Language is always English
TL = 'FR' #depending on the desired Target Language, this could be set, available abbr. choices are written in the introduction paragraph
en_fr = pd.read_csv(CD+SL+'-'+TL+'/'+SL+'-'+TL+'.txt', sep='\t', header=None, dtype=str)[[0,1]].rename(columns = {0:SL, 1:TL}).iloc[0:1000]
en_fr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   EN      1000 non-null   object
 1   FR      1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


In [5]:
CD = '../input/paralel-translation-corpus-in-22-languages/' #initially there were 22 languages, two more added afterwards :)
SL = 'EN' #this is a constant and should not be changed, i.e. Source Language is always English
TL = 'NL' #depending on the desired Target Language, this could be set, available abbr. choices are written in the introduction paragraph
en_nl = pd.read_csv(CD+SL+' -'+TL+'/'+SL+' -'+TL+'.txt', sep='\t', header=None, dtype=str)[[0,1]].rename(columns = {0:SL, 1:TL}).iloc[0:1000]
en_nl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   EN      1000 non-null   object
 1   NL      1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


In [6]:
en_fr = en_fr.drop_duplicates(subset="EN",keep="last",inplace=False,ignore_index=True)
en_fr = en_fr.drop_duplicates(subset="FR",keep="last",inplace=False,ignore_index=True)

In [7]:
en_nl = en_nl.drop_duplicates(subset="EN",keep="last",inplace=False,ignore_index=True)
en_nl = en_nl.drop_duplicates(subset="NL",keep="last",inplace=False,ignore_index=True)

In [8]:
nl_fr = en_nl.merge(en_fr,how="inner",on="EN")
nl_fr = nl_fr.drop(columns="EN")
nl_fr["FR"] = nl_fr["FR"].apply(lambda x: "[START] "+x+" [END]")
nl_fr

,NL,FR
0,Verordening (EG) nr. 1788/2004 van de Commissie,[START] Règlement (CE) no 1788/2004 de la Comm...
1,van 15 oktober 2004,[START] du 15 octobre 2004 [END]
2,tot vaststelling van de minimumverkoopprijzen ...,[START] fixant les prix minimaux de vente du b...
3,"DE COMMISSIE VAN DE EUROPESE GEMEENSCHAPPEN,",[START] LA COMMISSION DES COMMUNAUTÉS EUROPÉEN...
4,Gelet op het Verdrag tot oprichting van de Eur...,[START] vu le traité instituant la Communauté ...
...,...,...
942,De toepassing van deze regelen op de huidige m...,[START] L'application de ces modalités à la si...
943,De restituties bij de uitvoer van de in artike...,[START] Les restitutions à l'exportation des p...
944,Verordening laatstelijk gewijzigd bij Verorden...,[START] Règlement modifié en dernier lieu par ...
945,"PB L 78 van 31.3.1972, blz. 1.","[START] JO L 78 du 31.3.1972, p. 1. [END]"


In [9]:
val_number = int(0.2*nl_fr.shape[0])
nl_fr = nl_fr.sample(frac=1)
train_pairs = nl_fr.iloc[:-val_number]
val_pairs = nl_fr.iloc[-val_number:]

In [10]:
val_pairs.shape

(189, 2)

In [11]:
import string
import re
strip_chars = string.punctuation
f"[{re.escape(strip_chars)}]"

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\[\\\\\\]\\^_`\\{\\|\\}\\~]'

In [12]:
def text_standarization(_text):
    text_lowercase = tf.strings.lower(_text)
    return tf.strings.regex_replace(text_lowercase,f"[{re.escape(strip_chars)}]","")

In [13]:
vocab_size = 10000
sequence_length = 20

In [14]:
in_vectorization = tf.keras.layers.TextVectorization(max_tokens=vocab_size,
                                                    output_mode="int",
                                                    output_sequence_length=sequence_length)
# This will be used for the conversion of the prediction to translated ntext
out_vectorization = tf.keras.layers.TextVectorization(max_tokens=vocab_size,
                                                      output_mode="int",
                                                      output_sequence_length=sequence_length+1,
                                                      standardize=text_standarization)

2022-12-23 15:13:10.655053: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-23 15:13:10.656039: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-23 15:13:10.983555: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-23 15:13:10.984539: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-23 15:13:10.985462: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [15]:
in_vectorization.adapt(train_pairs["NL"])
out_vectorization.adapt(train_pairs["FR"])

2022-12-23 15:13:16.038791: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [ ]:
batch_size = 64
def format_dataset(_in,_out):
    _in = in_vectorization(_in)
    _out = out_vectorization(_out)
    return ((_in,_out[:,:-1]),
            _out[:,1:])

def make_dataset(_in,_out):
    dataset = tf.data.Dataset.from_tensor_slices((_in,_out))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset,num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

In [17]:
train_ds = make_dataset(train_pairs["NL"],train_pairs["FR"])
val_ds = make_dataset(val_pairs["NL"],val_pairs["FR"])

In [18]:
print(list(val_ds.as_numpy_iterator())[2])

({'in language': array([[1220,    8,    2, ...,    0,    0,    0],
       [  23, 1528,  251, ...,   15,    7, 3579],
       [   1,    0,    0, ...,    0,    0,    0],
       ...,
       [   6, 1496,    9, ...,    5,    1,    0],
       [   1,    8,    0, ...,    0,    0,    0],
       [  53,  263,    3, ...,    1,    3,    2]]), 'out language': array([[   4, 7676,  657, ...,    0,    0,    0],
       [   4,   36,  878, ...,  260, 1752,  100],
       [   4, 8208,    2, ...,    0,    0,    0],
       ...,
       [   4,    6,   82, ...,  860,    1,  632],
       [   4, 3847, 9098, ...,    0,    0,    0],
       [   4,  344,  585, ...,   26,   24,    1]])}, array([[7676,  657,    7, ...,    0,    0,    0],
       [  36,  878,    1, ..., 1752,  100, 1187],
       [8208,    2,    1, ...,    0,    0,    0],
       ...,
       [   6,   82, 6055, ...,    1,  632,    1],
       [3847, 9098,   14, ...,    0,    0,    0],
       [ 344,  585, 3772, ...,   24,    1,   11]]))


In [19]:
embed_dim = 256
latent_dim = 1024

In [20]:
source = tf.keras.Input(shape=(None,), dtype="int64", name="in language")
x = tf.keras.layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(latent_dim), merge_mode="sum")(x)

In [21]:
past_target = tf.keras.Input(shape=(None,), dtype="int64", name="out language")
x = tf.keras.layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = tf.keras.layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = tf.keras.layers.Dropout(0.5)(x)
target_next_step = tf.keras.layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = tf.keras.Model([source, past_target], target_next_step)

In [22]:
seq2seq_rnn.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
in language (InputLayer)        [(None, None)]       0                                            
__________________________________________________________________________________________________
out language (InputLayer)       [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 256)    2560000     in language[0][0]                
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    2560000     out language[0][0]               
______________________________________________________________________________________________

In [23]:
seq2seq_rnn.compile(optimizer="rmsprop",
                    loss="sparse_categorical_crossentropy",
                    metrics=["accuracy"])

In [24]:
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/5


2022-12-22 12:32:13.660060: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


35863/35863 [==============================] - 3066s 85ms/step - loss: 3.1207 - accuracy: 0.4001 - val_loss: 2.8807 - val_accuracy: 0.4429
Epoch 2/5
35863/35863 [==============================] - 3017s 84ms/step - loss: 3.0906 - accuracy: 0.4045 - val_loss: 3.0827 - val_accuracy: 0.4000
Epoch 3/5
35863/35863 [==============================] - 3007s 84ms/step - loss: 3.5669 - accuracy: 0.3213 - val_loss: 3.7296 - val_accuracy: 0.2936
Epoch 4/5
 4439/35863 [==>...........................] - ETA: 39:57 - loss: 3.9423 - accuracy: 0.2716

KeyboardInterrupt: 

In [25]:
import numpy as np
import random

out_vocab = out_vectorization.get_vocabulary()
out_index_lookup = dict(zip(range(len(out_vocab)), out_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = in_vectorization([input_sentence])
    decoded_sentence = "[START]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = out_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict([tokenized_input_sentence, tokenized_target_sentence])
        
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = out_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[END]":
            break
    return decoded_sentence

test_in_texts = val_pairs["NL"]
for _i in range(5):
    try:
        input_sentence = random.choice(test_in_texts)
        print("-")
        print(input_sentence)
        print(decode_sequence(input_sentence))
    except:
        print("error in example ",_i)

error in example  0
error in example  1
error in example  2
error in example  3
error in example  4


In [26]:
class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        
        self.attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        
        self.dense_proj = tf.keras.Sequential([tf.keras.layers.Dense(dense_dim, activation="relu"),
                                                     tf.keras.layers.Dense(embed_dim),])
        
        self.layernorm_1 = tf.keras.layers.LayerNormalization()
        self.layernorm_2 = tf.keras.layers.LayerNormalization()

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]  
        attention_output = self.attention(inputs,inputs,attention_mask=mask)
        proj_input = self.layernorm_1(inputs+attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


In [27]:
class TransformerDecoder(tf.keras.layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        
        self.attention_1 = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        
        self.dense_proj = tf.keras.Sequential([tf.keras.layers.Dense(dense_dim, activation="relu"),
                                               tf.keras.layers.Dense(embed_dim),])
        self.layernorm_1 = tf.keras.layers.LayerNormalization()
        self.layernorm_2 = tf.keras.layers.LayerNormalization()
        self.layernorm_3 = tf.keras.layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat([tf.expand_dims(batch_size, -1),
                          tf.constant([1, 1], dtype=tf.int32)],
                         axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(query=inputs,
                                              value=inputs,
                                              key=inputs,
                                              attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(query=attention_output_1,
                                              value=encoder_outputs,
                                              key=encoder_outputs,
                                              attention_mask=padding_mask,)
        attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)


In [28]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = tf.keras.layers.Embedding(input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = tf.keras.layers.Embedding(input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({"output_dim": self.output_dim,
                       "sequence_length": self.sequence_length,
                       "input_dim": self.input_dim,})
        return config

In [29]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = tf.keras.Input(shape=(None,), dtype="int64", name="in language")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = tf.keras.Input(shape=(None,), dtype="int64", name="out language")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = tf.keras.layers.Dropout(0.5)(x)
decoder_outputs = tf.keras.layers.Dense(vocab_size, activation="softmax")(x)
transformer = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [30]:
transformer.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
in language (InputLayer)        [(None, None)]       0                                            
__________________________________________________________________________________________________
out language (InputLayer)       [(None, None)]       0                                            
__________________________________________________________________________________________________
positional_embedding (Positiona (None, None, 256)    2565120     in language[0][0]                
__________________________________________________________________________________________________
positional_embedding_1 (Positio (None, None, 256)    2565120     out language[0][0]               
____________________________________________________________________________________________

In [31]:
transformer.compile(optimizer="rmsprop",
                    loss="sparse_categorical_crossentropy",
                    metrics=["accuracy"])


In [ ]:
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
35863/35863 [==============================] - 2161s 60ms/step - loss: 2.9103 - accuracy: 0.4500 - val_loss: 2.5322 - val_accuracy: 0.5076
Epoch 2/30
35863/35863 [==============================] - 2153s 60ms/step - loss: 2.6167 - accuracy: 0.5041 - val_loss: 2.4128 - val_accuracy: 0.5280
Epoch 3/30
35863/35863 [==============================] - 2153s 60ms/step - loss: 2.5388 - accuracy: 0.5179 - val_loss: 2.3625 - val_accuracy: 0.5376
Epoch 4/30
35863/35863 [==============================] - 2154s 60ms/step - loss: 2.4987 - accuracy: 0.5251 - val_loss: 2.3324 - val_accuracy: 0.5442
Epoch 5/30
35863/35863 [==============================] - 2151s 60ms/step - loss: 2.4712 - accuracy: 0.5299 - val_loss: 2.3182 - val_accuracy: 0.5466
Epoch 6/30
35863/35863 [==============================] - 2153s 60ms/step - loss: 2.4499 - accuracy: 0.5336 - val_loss: 2.3147 - val_accuracy: 0.5459
Epoch 7/30
35863/35863 [==============================] - 2152s 60ms/step - loss: 2.4353 - accuracy:

In [1]:
import numpy as np
out_vocab = out_vectorization.get_vocabulary()
out_index_lookup = dict(zip(range(len(out_vocab)), out_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = in_vectorization([input_sentence])
    decoded_sentence = "[START]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = out_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = out_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[END]":
            break
    return decoded_sentence

test_in_texts = nl_fr["NL"]
for _ in range(20):
    input_sentence = random.choice(test_in_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

NameError: name 'out_vectorization' is not defined